<a href="https://colab.research.google.com/github/insilicomab/Pytorch_official_tutorials_JP/blob/main/finetuning_torchvision_models_tutorial_jp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

In [ ]:
%matplotlib inline

このチュートリアルでは、1000クラスのImagenetデータセットで事前学習されたtorchvisionモデルのファインチューニングと特徴抽出の方法をより深く見ていきます。このチュートリアルでは、いくつかの最新のCNNアーキテクチャの扱い方を深く見ていき、どのPyTorchモデルでもファインチューニングできるような直感を構築していきます。各モデルのアーキテクチャは異なるので、すべてのシナリオで機能する定型的な微調整コードはありません。むしろ、研究者は既存のアーキテクチャを見て、各モデルに対してカスタム調整を行う必要があります。
  
  
このドキュメントでは、2種類の転移学習を行う。すなわち、ファインチューニングと特徴抽出である。ファインチューニングでは、事前に学習したモデルから始めて、新しいタスクに対してモデルの全てのパラメータを更新します。特徴抽出では、事前に学習したモデルを用いて、最終層の重みのみを更新し、そこから予測値を導きます。これは、事前学習されたCNNを固定された特徴抽出器として使い、出力層のみを変更することから特徴抽出と呼ばれています。転移学習に関するより詳しい技術的な情報はこちら（<https://cs231n.github.io/transfer-learning/>）とこちら（<https://ruder.io/transfer-learning/>）をご覧ください。


一般に、どちらの転移学習法も同じようないくつかのステップを踏みます：

*   学習済みモデルを初期化する
*   新しいデータセットのクラス数と同じ数の出力を持つように、最終層を再形成する。
*   最適化アルゴリズムに対して、学習中に更新したいパラメータを定義する
*   学習ステップを実行



In [ ]:
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.11.0+cu113
Torchvision Version:  0.12.0+cu113


# Inputs

ここでは、実行のために変更するすべてのパラメータを示します。ここでは、ここからダウンロードできる *hymenoptera_data* データセットを使用します <https://download.pytorch.org/tutorial/hymenoptera_data.zip>。このデータセットにはハチとアリの2つのクラスがあり、独自のカスタムデータセットを書くのではなく、`ImageFolder` <https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.ImageFolder>データセットを使うことができるような構造になっています。データをダウンロードし、`data_dir`入力にデータセットのルートディレクトリを設定する。`model_name`入力は使いたいモデルの名前で、このリストから選択する必要がある。

### `[resnet, alexnet, vgg, squeezenet, densenet, inception]`

`num_classes` はデータセットのクラス数、`batch_size` は学習に使用するバッチサイズ、マシンの能力に応じて調整可能、`num_epochs` は実行したい学習エポックの数、`feature_extract` はファインチューニングか特徴抽出かを定義するブール型です。`feature_extract = False` の場合、モデルはファインチューニングされ、すべてのモデルパラメータが更新されます。`feature_extract = True`の場合、最後のレイヤーのパラメーターだけが更新され、他のパラメーターは固定されたままです。

In [ ]:
# トップレベルのデータディレクトリ
# ここでは、ディレクトリの形式がImageFolderの構造に準拠するものとする。
data_dir = "./data/hymenoptera_data"

# 選べるモデル [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "squeezenet"

# データセットに含まれるクラス数
num_classes = 2

# 学習用バッチサイズ（メモリ容量により変更可）
batch_size = 8

# 学習するエポック数 
num_epochs = 15

# 特徴抽出のためのフラグ
# Falseの場合、モデル全体を微調整し、
# Trueの場合、リシェイプされたレイヤーパラメータのみを更新します
feature_extract = True

# ヘルパー関数

モデルを調整するコードを書く前に、いくつかのヘルパー関数を定義しておきましょう。  

# モデル学習・検証用コード


`train_model` 関数は、与えられたモデルの学習と検証を処理します。入力として、PyTorchモデル、データロードの辞書、損失関数、オプティマイザ、学習と検証を行うエポック数、モデルがInceptionモデルであるかどうかのブール型フラグを受け取ります。*is_inception*フラグは、Inception v3モデルに対応するために使用されます。このアーキテクチャでは、補助出力を使用し、モデル全体の損失は、ここで説明するように、補助出力と最終出力の両方を尊重するからです。この関数は、指定されたエポック数で学習を行い、各エポックの後に完全な検証ステップを実行します。また，（検証精度の点で）最も性能の良いモデルを追跡し，トレーニングの終了時に最も性能の良いモデルを返します．各エポックの後、学習と検証の精度が表示されます。

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # 各エポックには学習と検証の段階がある
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # モデルを学習モードに設定する
            else:
                model.eval()   # モデルを評価モードに設定する

            running_loss = 0.0
            running_corrects = 0

            # データを繰り返し処理する
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # パラメータの勾配をゼロにする
                optimizer.zero_grad()

                # forward
                # 学習モードのみの場合は履歴を追跡する
                with torch.set_grad_enabled(phase == 'train'):
                    # モデル出力の取得と損失額の算出
                    # トレーニングでは補助出力があるため、inceptionの場合は特殊なケース。
                    # トレーニングモードでは最終出力と補助出力の合計で損失を計算するが、
                    # テストでは最終出力のみを考慮する
                    if is_inception and phase == 'train':
                        # https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize （学習モードのみ）
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # モデルをディープコピーする
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # 最適なモデルの重みをロードする
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

# モデルパラメータの.requests_grad属性の設定


このヘルパー関数は、特徴抽出を行う際に、モデルのパラメータの `.requires_grad` 属性を False に設定します。デフォルトでは、学習済みモデルをロードする際、全てのパラメータは `.requires_grad=True` となります。これは、ゼロから学習する場合や微調整を行う場合には問題ありません。しかし、特徴抽出を行い、新しく初期化されたレイヤーに対してのみ勾配を計算したい場合、他のすべてのパラメータは勾配を必要としないようにします。これは後々より意味を持つようになる。

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

# ネットワークの初期化と再形成

さて、最も興味深い部分です。ここでは、各ネットワークのリシェイプを処理します。これは自動的な手順ではなく、各モデルに固有であることに注意されたい。CNNモデルの最終層、それはしばしばFC層であるが、データセットの出力クラスの数と同じ数のノードを持っていることを思い出してほしい。すべてのモデルはImagenetで事前学習されているので、それらはすべてサイズ1000の出力層を持っており、各クラスに1つのノードがあります。ここでのゴールは、最後の層を以前と同じ数の入力を持ち、かつデータセットのクラス数と同じ数の出力を持つように再形成することです。以下の節では、各モデルのアーキテクチャを個別に変更する方法について説明します。しかしその前に、微調整と特徴抽出の違いについて一つ重要なことがある。

特徴抽出の場合、我々は最後の層のパラメータのみを更新したい、言い換えれば、整形する層のパラメータのみを更新したい。したがって、変更しないパラメータの勾配を計算する必要はないので、効率化のために `.requires_grad` 属性を `False` に設定します。これは重要なことで、デフォルトではこのアトリビュートは`True`に設定されています。そして、新しいレイヤーを初期化すると、デフォルトで新しいパラメーターは`.requests_grad=True`となり、新しいレイヤーのパラメーターだけが更新されます。微調整を行う際には、全ての`.required_grad`をデフォルトの`True`に設定したままにしておくことができます。

最後に、inception_v3は入力サイズに(299,299)を要求していますが、他のモデルは(224,224)を要求していることに注意してください。

# Resnet

Resnetは、Deep Residual Learning for Image Recognitionという論文で紹介されました。Resnet18、Resnet34、Resnet50、Resnet101、Resnet152など、サイズの異なるいくつかのバリエーションがあり、いずれもtorchvisionモデルから入手可能である。ここでは、我々のデータセットが小さく、2クラスしかないため、Resnet18を使用します。モデルを印刷すると、以下のように最後の層が完全連結層であることがわかります。

### `(fc): Linear(in_features=512, out_features=1000, bias=True)`

したがって、`model.fc`を512個の入力特徴と2個の出力特徴を持つLinear層に再初期化する必要がある。

### `model.fc = nn.Linear(512, num_classes)`

# Alexnet

Alexnetは論文ImageNet Classification with Deep Convolutional Neural Networksで紹介され、ImageNetデータセットで最初に大成功したCNNでした。モデルのアーキテクチャを表示すると、モデルの出力は分類器の6層目から来ることが分かります



```
(classifier): Sequential(
    ...
    (6): Linear(in_features=4096, out_features=1000, bias=True)
 )
```



我々のデータセットでモデルを使うために、このレイヤーを次のように再初期化する。

### `model.classifier[6] = nn.Linear(4096,num_classes)`

# VGG

VGGはVery Deep Convolutional Networks for Large-Scale Image Recognitionという論文で紹介されました。Torchvisionは様々な長さの8つのバージョンのVGGを提供しており、中にはバッチ正規化レイヤーを持つものもあります。ここでは、バッチ正規化のあるVGG-11を使用します。出力層はAlexnetと同様、すなわち



```
(classifier): Sequential(
    ...
    (6): Linear(in_features=4096, out_features=1000, bias=True)
 )
```



そこで、同じ手法で出力層を修正します

### `model.classifier[6] = nn.Linear(4096,num_classes)`

# Squeezenet

Squeeznetのアーキテクチャは論文SqueezeNetで説明されています。50倍少ないパラメータと<0.5MBのモデルサイズでAlexNetレベルの精度を実現し、ここに示した他のどのモデルとも異なる出力構造を使用しています。Torchvisionには2つのバージョンのSqueezenetがあり、私たちはバージョン1.0を使用しています。出力は、分類器の第1層である1x1畳み込み層から得られます。



```
(classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Conv2d(512, 1000, kernel_size=(1, 1), stride=(1, 1))
    (2): ReLU(inplace)
    (3): AvgPool2d(kernel_size=13, stride=1, padding=0)
 )
```



ネットワークを修正するために、Conv2d層を再初期化し、深さ2の出力特徴マップを次のようにする。

### ```model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))```

# Densenet

DensenetはDensely Connected Convolutional Networksという論文で紹介されました。Torchvisionには4種類のDensenetがありますが、ここではDensenet-121のみを使用します。出力層は1024個の入力特徴量を持つ線形層である。

### `(classifier): Linear(in_features=1024, out_features=1000, bias=True)`

ネットワークを再形成するために，分類器の線形層を以下のように再初期化する．

### `model.classifier = nn.Linear(1024, num_classes)`

# Inception v3

最後に、Inception v3は、Rethinking the Inception Architecture for Computer Visionで初めて紹介されました。このネットワークは、トレーニング時に2つの出力レイヤーを持つことが特徴です。2つ目の出力は補助出力と呼ばれ、ネットワークのAuxLogits部分に含まれる。最終出力はネットワークの末尾にある線形層である。なお、テスト時には最終出力のみを考慮する。ロードされたモデルの補助出力と最終出力は次のように出力される。



```
(AuxLogits): InceptionAux(
    ...
    (fc): Linear(in_features=768, out_features=1000, bias=True)
 )
 ...
(fc): Linear(in_features=2048, out_features=1000, bias=True)
```



このモデルをファインチューニングするためには、両方のレイヤーを再形成する必要があります。これは次のようにして行います。



```
model.AuxLogits.fc = nn.Linear(768, num_classes)
model.fc = nn.Linear(2048, num_classes)
```



多くのモデルは似たような出力構造を持っていますが、それぞれを少し異なるように処理しなければならないことに注意してください。また、整形されたネットワークのプリントモデルアーキテクチャを確認し、出力特徴の数がデータセットのクラスの数と同じであることを確認します。

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # このif文で設定されるこれらの変数を初期化します
    # これらの変数はそれぞれモデルに依存します
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
        input_size = 224

    elif model_name == "inception":
        """ Inception v3 
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # 補助出力
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # 最終出力
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_ft, input_size

# この実行のためのモデルを初期化する
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# インスタンス化したばかりのモデルを表示する
print(model_ft)

Downloading: "https://download.pytorch.org/models/squeezenet1_0-b66bff10.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_0-b66bff10.pth


  0%|          | 0.00/4.78M [00:00<?, ?B/s]

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

# データのロード

入力サイズがわかったので、データ変換、画像データセット、データローダを初期化することができます。ここで説明するように、モデルはハードコードされた正規化値で事前学習されていることに注意してください（<https://pytorch.org/docs/master/torchvision/models.html>）。

In [ ]:
# 学習のためのデータの水増しと正規化
# 検証のための正規化
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# 学習データセットと検証データセットの作成
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# train dataloadersとvalidation dataloadersの作成
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# GPUが利用可能かどうかを検出する
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


FileNotFoundError: ignored

# オプティマイザーの作成

モデル構造が正しいので、ファインチューニングと特徴抽出のための最終ステップは、必要なパラメータのみを更新するオプティマイザを作成することです。事前学習されたモデルをロードした後、再形成する前に、`feature_extract=True`の場合、パラメータの`.requests_grad`属性を全て`False`に手動で設定したことを思い出してください。そして、再初期化されたレイヤーのパラメータは、デフォルトで` .requires_grad=True` になっています。これで、`.requests_grad=True`のパラメータはすべて最適化する必要があることがわかりました。次に、そのようなパラメータのリストを作成し、このリストを SGD アルゴリズムのコンストラクタに入力します。

これを確認するために、出力されたパラメータを確認して学習します。ファインチューニングを行う場合、このリストは長くなり、すべてのモデルパラメータを含む必要があります。しかし、特徴抽出時には、このリストは短く、再形成された層の重みとバイアスのみを含むべきである。

In [ ]:
# モデルをGPUに送る
model_ft = model_ft.to(device)

# このランで最適化/更新するパラメータを収集します
# ファインチューニングを行う場合は、すべてのパラメータを更新する
# しかし、特徴抽出を行う場合は、先ほど初期化したパラメータ、
# つまり requires_grad = True のパラメータのみを更新する
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# すべてのパラメータが最適化されていることを確認する
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

# トレーニングおよび検証ステップを実行する

最後に、モデルの損失を設定し、設定したエポック数で学習と検証の機能を実行します。エポック数によっては、このステップにCPUの時間がかかることがあります。また、デフォルトの学習率はすべてのモデルに最適ではないので、最大の精度を得るためには、各モデルを個別に調整する必要があります。

In [ ]:
# 損失関数を設定する
criterion = nn.CrossEntropyLoss()

# 学習と評価
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

# ゼロからトレーニングされたモデルとの比較

面白いことに、転移学習を用いない場合、モデルがどのように学習するのかを見てみましょう。微調整と特徴抽出の性能はデータセットに大きく依存しますが、一般的にどちらの転移学習法も、ゼロから学習したモデルに対して、学習時間と全体的な精度の面で有利な結果を生み出します。

In [ ]:
# この実行に使用されるモデルの事前トレーニングされていないバージョンを初期化します
scratch_model,_ = initialize_model(model_name, num_classes, feature_extract=False, use_pretrained=False)
scratch_model = scratch_model.to(device)
scratch_optimizer = optim.SGD(scratch_model.parameters(), lr=0.001, momentum=0.9)
scratch_criterion = nn.CrossEntropyLoss()
_,scratch_hist = train_model(scratch_model, dataloaders_dict, scratch_criterion, scratch_optimizer, num_epochs=num_epochs, is_inception=(model_name=="inception"))

# 転移学習法とゼロから学習したモデルについて、
# 検証精度対学習エポック数の学習曲線をプロットしたもの
ohist = []
shist = []

ohist = [h.cpu().numpy() for h in hist]
shist = [h.cpu().numpy() for h in scratch_hist]

plt.title("Validation Accuracy vs. Number of Training Epochs")
plt.xlabel("Training Epochs")
plt.ylabel("Validation Accuracy")
plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
plt.plot(range(1,num_epochs+1),shist,label="Scratch")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.show()

# 最終的な考えと次に進むべき場所

他のモデルも動かしてみて、精度がどの程度になるかを見てみましょう。また、バックワードパスでは勾配の大部分を計算する必要がないため、特徴抽出にかかる時間が短くなっていることに注目してください。ここから先はいろいろなことが考えられます。あなたにもできるはずです。



*   このコードをより難しいデータセットで実行し，転移学習の利点をさらに見てみましょう． 
*   ここで説明した方法を使って、転移学習により、おそらく新しいドメイン（NLP、オーディオなど）の別のモデルを更新する。
*   モデルが完成したら、ONNXモデルとしてエクスポートしたり、ハイブリッドフロントエンドを使用してトレースすることで、より高速で最適化されたモデルを作成することができます。



